In [53]:
# imports
import sys
import os
import socket
import pandas as pd
import glob
import datetime as dt
from datetime import timedelta, datetime

In [54]:
# function to format dates
def formatdate(data):
    data['sysdate'] = data['sysdate'].str.replace('/', '')
    data['sysdate']= pd.to_datetime(data['sysdate'], format='%d%m%Y')
    data['eventdate']= data['eventdate'].str.replace('/', '')
    data['eventdate'] = pd.to_datetime(data['eventdate'], format='%d%m%Y')
    data['eventdate'].fillna(data['sysdate'], inplace = True)
    return data

# function to get latest record in a df of e_patids and dates
def getlatest(data):
    data = data[['e_patid', 'eventdate']].groupby(['e_patid']).max()
    return data

In [55]:
# read in cohort file
cohort = pd.read_parquet(os.getcwd() + '/cleaned_cohort_file.parquet')

In [56]:
path = "/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Linked Data/"

In [57]:
# get unique e_patids
cohort_e_patids = list(cohort.e_patid.unique())

In [58]:
#read in and merge all the hes patient files and search by cohort e_patid

hes_patient_files = glob.glob(path + "*patient*.txt")

patient_list = []
for filename in hes_patient_files:
    df = pd.read_csv(filename, sep="\t")
    df = df[df['e_patid'].isin(cohort_e_patids)]
    patient_list.append(df)

patient_merged = pd.concat(patient_list, axis=0)

/mnt/ebinstall/software/Anaconda3/2020.02/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (7,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/mnt/ebinstall/software/Anaconda3/2020.02/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (7,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [59]:
patient_merged.head()

,e_patid,e_pracid,e_gen_hesid,n_patid_hes,gen_ethnicity,match_rank,e_gen_mhds_id,n_patid_mh,pre_2011,post_2011,...,cause14,cause15,cause_neonatal1,cause_neonatal2,cause_neonatal3,cause_neonatal4,cause_neonatal5,cause_neonatal6,cause_neonatal7,cause_neonatal8
19,10002112,112,54425.0,1.0,White,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,10007093,93,853202.0,1.0,Unknown,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,10011120,20,542707.0,1.0,White,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,100163204,204,1054818.0,1.0,White,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,10017321,21,1332192.0,1.0,White,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
patient_merged.shape[0]

71879

In [61]:
len(cohort_e_patids)

25453

In [62]:
# drop duplicates for patient ethnicity and any where ethnicity is unknown
patient_ethnicities = patient_merged[['e_patid', 'gen_ethnicity']].drop_duplicates()
patient_ethnicities = patient_ethnicities.dropna()
patient_ethnicities = patient_ethnicities[patient_ethnicities.gen_ethnicity != 'Unknown']

In [63]:
len(patient_ethnicities.e_patid.unique())

20595

In [64]:
patient_ethnicities.shape[0]

20794

In [65]:
patient_ethnicities.e_patid.value_counts()

28508165     2
76832526     2
49300987     2
306962252    2
269243149    2
            ..
347218226    1
347172206    1
347145102    1
34714414     1
95428100     1
Name: e_patid, Length: 20595, dtype: int64

In [66]:
patient_ethnicities[patient_ethnicities.e_patid.isin([11105980, 734211177, 5368195])]

,e_patid,gen_ethnicity
10122,11105980,Bl_Afric
399921,5368195,Indian
580054,734211177,White
299541,5368195,White
1453,11105980,White
86417,734211177,Other


In [67]:
patient_ethnicities.gen_ethnicity.value_counts()

White         20189
Other           173
Bl_Carib        131
Indian           91
Oth_Asian        52
Mixed            42
Bl_Afric         33
Pakistani        29
Bl_Other         26
Chinese          17
Bangladesi       11
Name: gen_ethnicity, dtype: int64

In [68]:
# set any patient with two or more ethnicities to 'mixed' and shrink to less categories to match CPRD GOLD ones
ethn_e_patids = list(patient_ethnicities.e_patid.unique())
ethnicities_list = []
for i in ethn_e_patids:
    if patient_ethnicities[patient_ethnicities.e_patid == i].shape[0] > 1:
        ethnicity_1 = list(patient_ethnicities[
            patient_ethnicities.e_patid == i].gen_ethnicity)[0]
        ethnicity_2 = list(patient_ethnicities[
            patient_ethnicities.e_patid == i].gen_ethnicity)[1]
        if ethnicity_1 in [
            'Bl_Carib', 'Bl_Afric', 'Bl_Other'] and ethnicity_2 in [
            'Bl_Carib', 'Bl_Afric', 'Bl_Other']:
            ethnicities_list.append('Black')
        elif ethnicity_1 in [
            'Indian', 'Pakistani', 'Bangladesi'] and ethnicity_2 in [
            'Indian', 'Pakistani', 'Bangladesi']:
            ethnicities_list.append('S_Asian')
        else:
            ethnicities_list.append('Mixed')
    else:
        ethnicity = list(patient_ethnicities[
            patient_ethnicities.e_patid == i].gen_ethnicity)[0]
        if ethnicity in ['Bl_Carib', 'Bl_Afric', 'Bl_Other']:
            ethnicities_list.append('Black')
        elif ethnicity in ['Indian', 'Pakistani', 'Bangladesi']:
            ethnicities_list.append('S_Asian')
        elif ethnicity == 'Mixed':
            ethnicities_list.append('Mixed')
        elif ethnicity == 'White':
            ethnicities_list.append('White')
        else:
            ethnicities_list.append('Other')
        
ethnicities_df_out = pd.DataFrame({'e_patid': ethn_e_patids, 'Ethnicity': ethnicities_list})
ethnicities_df_out.head()

,e_patid,Ethnicity
0,10002112,White
1,10011120,White
2,100163204,White
3,10017321,White
4,100175164,White


In [69]:
ethnicities_df_out.Ethnicity.value_counts()

White      20046
Mixed        192
Black        139
Other        120
S_Asian       98
Name: Ethnicity, dtype: int64

In [70]:
# read in cprd ethnicity codes
ethnicity_codes = pd.read_excel(os.getcwd() + '/Risk Factor Codes/ethnicity.dta.xls')
ethnicity_codes.head()

,categoryname,category,readcode,readterm,medcode
0,White British (1),1,9S10.00,White British,12446
1,White British (1),1,9S13.00,White Scottish,26467
2,White British (1),1,9S14.00,Other white British ethnic group,26310
3,White British (1),1,9i00.00,White British - ethnic category 2001 census,98111
4,White British (1),1,9i20.00,English - ethnic category 2001 census,12352


In [71]:
path2 = "/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/"

In [72]:
# get all ethnicity med codes that aren't 'not specified'
ethnicity_med_codes = ethnicity_codes[ethnicity_codes['categoryname'] != 'Ethnic group not specified (21)']['medcode']

In [73]:
clinical_files = glob.glob(path2 + "*clinical*.txt") # list all files in the directory with "clinical" in filename

#read  all the clinical files in and add to a list, format and search for cohort patients
clinical_list = []
for filename in clinical_files:    
    df = pd.read_csv(filename, sep="\t")
    df = df[df['e_patid'].isin(cohort_e_patids)]
    df = formatdate(df)
    df = df[df['medcode'].isin(ethnicity_med_codes)][['e_patid', 'eventdate', 'medcode']]
    clinical_list.append(df)

clinical_merged = pd.concat(clinical_list, axis=0) # merge all the clinical files together

In [74]:
# get most recent CPRD primary care ethnicity date
print(clinical_ethnicity_codes.shape[0])

clin_ethn_e_patids = clinical_ethnicity_codes['e_patid'].unique()
last_clin_ethn_code = []
for i in clin_ethn_e_patids:
    last_clin_ethn_code.append(max(clinical_ethnicity_codes[clinical_ethnicity_codes['e_patid'] == i]['eventdate']))
    
last_ethn_code_dates_df = pd.DataFrame({'e_patid': clin_ethn_e_patids, 'eventdate': last_clin_ethn_code})

last_clinical_ethnicity_codes = last_ethn_code_dates_df.merge(
    right = clinical_ethnicity_codes, on = ['e_patid', 'eventdate'], how = 'inner').drop_duplicates()

print(last_clinical_ethnicity_codes.shape[0])

5916
4888


In [75]:
# merge to get the most recent codes
last_clinical_ethnicity_codes = last_clinical_ethnicity_codes.merge(
    right = ethnicity_codes[['medcode', 'category']], 
    on = ['medcode'], how = 'inner')

last_clinical_ethnicity_codes = last_clinical_ethnicity_codes.drop(columns = ['medcode']).drop_duplicates()

In [76]:
last_clinical_ethnicity_codes.category.value_counts()

1    4643
4      85
3      68
5      46
2      17
Name: category, dtype: int64

In [77]:
last_clinical_ethnicity_codes.e_patid.value_counts()

765150233    2
301172206    2
11788156     2
437834112    1
47821539     1
            ..
784021199    1
780786199    1
77644156     1
772983199    1
55617483     1
Name: e_patid, Length: 4856, dtype: int64

In [78]:
# map to same categories as HES
last_clinical_ethnicity_codes['Ethnicity'] = last_clinical_ethnicity_codes.category.map({1: 'White', 2: 'Mixed',
                                                                                        3: 'S_Asian', 4: 'Black',
                                                                                        5: 'Other'})
last_clinical_ethnicity_codes = last_clinical_ethnicity_codes.drop(columns = ['category'])

In [80]:
last_clinical_ethnicity_codes[last_clinical_ethnicity_codes['e_patid'].isin([765150233, 301172206, 11788156])]

,e_patid,eventdate,Ethnicity
4616,765150233,2008-04-03,Black
4707,765150233,2008-04-03,Mixed
4840,301172206,2008-07-02,Other
4841,301172206,2008-07-02,Mixed
4867,11788156,2013-07-10,Black
4872,11788156,2013-07-10,Other


In [81]:
# set patients with more than one ethnicity to mixed
for i in clin_ethn_e_patids:
    if last_clinical_ethnicity_codes[last_clinical_ethnicity_codes.e_patid == i].shape[0] > 1:
        dfi = last_clinical_ethnicity_codes[last_clinical_ethnicity_codes.e_patid == i]
        index = dfi.index
        last_clinical_ethnicity_codes = last_clinical_ethnicity_codes.drop(index[1:])
        last_clinical_ethnicity_codes.loc[index[0], 'Ethnicity'] = 'Mixed'

In [82]:
last_clinical_ethnicity_codes.e_patid.value_counts()

10515182     1
508049131    1
486541112    1
47821539     1
472044131    1
            ..
784021199    1
780786199    1
77644156     1
772983199    1
55617483     1
Name: e_patid, Length: 4856, dtype: int64

In [83]:
# merge CPRD and HES ethnicities taking CPRD where possible and HES where missing, any nans as unspecified
merged_ethnicities = []
for i in cohort_e_patids:
    if i in clin_ethn_e_patids:
        merged_ethnicities.append(
            list(last_clinical_ethnicity_codes[last_clinical_ethnicity_codes.e_patid == i].Ethnicity)[0])
    elif i in ethn_e_patids:
        merged_ethnicities.append(list(ethnicities_df_out[ethnicities_df_out.e_patid == i].Ethnicity)[0])
    else:
        merged_ethnicities.append('Unspecified')
        
merged_ethnicities_df = pd.DataFrame({'e_patid': cohort_e_patids, 'Ethnicity': merged_ethnicities})
merged_ethnicities_df.head()

,e_patid,Ethnicity
0,24117254,White
1,519045131,White
2,734003216,White
3,10485798,White
4,630194220,White


In [86]:
merged_ethnicities_df.Ethnicity.value_counts()

White          20257
Unspecified     4669
Black            158
Other            130
Mixed            123
S_Asian          116
Name: Ethnicity, dtype: int64

In [87]:
# Output to parquet
merged_ethnicities_df.to_parquet(os.getcwd() + '/Merged_Ethnicities.parquet')